In [1]:
import pycurl
import io
import pandas as pd
import datetime
from datetime import date

c = pycurl.Curl()
f = open("AAPL_ticker_list", "r")
#f = open("ticker_list_short", "r")

In [2]:
# initialize list of dicts to store date, before creating frame
px_rows = []

for ticker in f:
    
    buffer = io.BytesIO()
    
    ticker = ticker.strip()
    print(ticker)
    local_url = "https://query1.finance.yahoo.com/v7/finance/download/"+ticker+"?period1=378604800&period2=1586476800&interval=1d&events=history"
    print(local_url)

    c.setopt(c.URL, local_url)
    c.setopt(c.WRITEDATA, buffer)
    c.perform()
    c.getinfo(pycurl.HTTP_CODE)
    
    full_text = ""
    full_text = buffer.getvalue().decode()
    buffer.flush()
    buffer.close()
    
    ## for each line, stripping out fields to create dict
    ## loading dict in bulk into frame is much more efficient than indexing a frame repeatedly
    
    lines = full_text.splitlines()
    for line in lines[1:]:
        #print("here",line.split(',')[1],len(line.split(',')[1]))
        if len(line.split(',')[0]) and len(line.split(',')[1]) and len(line.split(',')[2]) and len(line.split(',')[3]) and len(line.split(',')[4]) and len(line.split(',')[5]) and len(line.split(',')[6]) and line.split(',')[0] != 'null' and line.split(',')[1] != 'null' and line.split(',')[2] != 'null' and line.split(',')[3] != 'null' and line.split(',')[4] != 'null' and line.split(',')[5] != 'null' and line.split(',')[6] != 'null':
            #local_date = datetime.datetime.strptime(line.split(',')[0], "%Y-%m-%d").strftime("%d-%m-%Y")
            local_date = datetime.datetime.strptime(line.split(',')[0], "%Y-%m-%d").date()
            px_dict = {
                "ticker": ticker,
                "date": local_date,
                "open": float(line.split(',')[1]),
                "high": float(line.split(',')[2]),
                "low": float(line.split(',')[3]),
                "close": float(line.split(',')[4]),
                "adj_close": float(line.split(',')[5]),
                "volume": int(line.split(',')[6]),
                "delta": (float(line.split(',')[4])) - float((line.split(',')[1])) 
            }
            px_rows.append(px_dict)

master_px_frame = pd.DataFrame(px_rows)

AAPL
https://query1.finance.yahoo.com/v7/finance/download/AAPL?period1=378604800&period2=1586476800&interval=1d&events=history


In [3]:
master_px_frame.describe()

,adj_close,close,delta,high,low,open,volume
count,9651.000000,9651.000000,9651.000000,9651.000000,9651.000000,9651.000000,9.651000e+03
mean,31.544251,33.637788,0.012140,33.966416,33.284887,33.625648,8.790460e+07
std,57.552180,59.265395,0.943662,59.804593,58.662455,59.206608,8.614811e+07
min,0.155638,0.196429,-17.939988,0.198661,0.196429,0.198661,3.472000e+05
25%,0.977499,1.162946,-0.035714,1.195312,1.133929,1.160714,3.442195e+07
50%,1.506051,1.794643,0.000000,1.821429,1.758929,1.794643,5.936450e+07
75%,33.546681,38.646428,0.041071,39.007143,38.306429,38.685715,1.088294e+08
max,327.200012,327.200012,16.529999,327.850006,323.350006,324.739990,1.855410e+09


In [4]:
master_px_frame.shape

(9651, 9)

In [10]:
master_px_frame.dtypes

adj_close    float64
close        float64
date          object
delta        float64
high         float64
low          float64
open         float64
ticker        object
volume         int64
dtype: object

In [11]:
from datetime import date
start_date = date(2020, 4, 1)
#end_date = '2020-04-10'
master_px_frame[(master_px_frame['date'] >= start_date)]

,adj_close,close,date,delta,high,low,open,ticker,volume
9644,240.910004,240.910004,2020-04-01,-5.589996,248.720001,239.130005,246.500000,AAPL,44054600
9645,244.929993,244.929993,2020-04-02,4.589997,245.149994,236.899994,240.339996,AAPL,41483500
9646,241.410004,241.410004,2020-04-03,-1.389999,245.699997,238.970001,242.800003,AAPL,32470000
9647,262.470001,262.470001,2020-04-06,11.570007,263.109985,249.380005,250.899994,AAPL,50455100
9648,259.429993,259.429993,2020-04-07,-11.369995,271.700012,259.000000,270.799988,AAPL,50721800
9649,266.070007,266.070007,2020-04-08,3.330017,267.369995,261.230011,262.739990,AAPL,42223800
9650,267.989990,267.989990,2020-04-09,-0.710022,270.070007,264.700012,268.700012,AAPL,40529100
